# Challenge

- The dataset above has average house prices for various regions in California (each row corresponds to a single region).
- Suppose, however, that you could not observe the "poorest" regions; that is, regions where the MedInc is smaller than $25,000.
- Can you build a model that will predict avg_price "well" on the poorest regions using only the observed data?

That is, pretend you only have "observed_data", build an ML model that predicts "well" on the "unobserved_data"

ozellikle poor region datasini ayirmis

poor region daki house price i tahmin etmek istiyor

ben istemesemde linear regression train ettim

train data observed data oldu, unobserved data ise test oldu, sonuc felaket

adamin kafasinda regression dicontuinuity var

median income scoring variable olarak dusunuyor

extrapolation yaparak poor region tahmin etmek istiyordu

In [3]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing

# Load the California Housing dataset
data = fetch_california_housing()

In [4]:
# The features are stored in data.data, and the target values are in data.target
X, y = data.data, data.target

# Convert to a DataFrame
df = pd.DataFrame(data.data, columns=data.feature_names)
df['avg_price'] = data.target

# split by MedInc (measured in tens of thousands of US Dollars)
observed_data = df.query('MedInc > 2.5').copy()
unobserved_data = df.query('MedInc <= 2.5').copy()

df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,avg_price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [5]:
df.shape

(20640, 9)

In [6]:
observed_data.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,avg_price
count,15835.000000,15835.000000,15835.000000,15835.000000,15835.000000,15835.000000,15835.000000,15835.000000,15835.000000
mean,4.465175,27.920998,5.694863,1.084786,1451.798926,3.056434,35.559617,-119.591758,2.327992
std,1.770240,12.683595,2.270236,0.444644,1158.651117,11.832547,2.063182,1.998768,1.138569
min,2.500800,1.000000,0.846154,0.333333,8.000000,0.692308,32.550000,-124.350000,0.149990
25%,3.218800,17.000000,4.763328,1.001742,802.000000,2.434969,33.910000,-121.870000,1.497000
50%,4.033200,28.000000,5.482270,1.042151,1176.000000,2.793233,34.230000,-118.480000,2.072000
75%,5.169700,36.000000,6.253626,1.089474,1741.000000,3.192982,37.675000,-117.980000,2.907000
max,15.000100,52.000000,132.533333,34.066667,35682.000000,1243.333333,41.880000,-114.570000,5.000010


In [7]:
observed_data.corr()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,avg_price
MedInc,1.000000,-0.078814,0.325226,-0.049465,-0.033198,0.025034,-0.063649,-0.006102,0.641759
HouseAge,-0.078814,1.000000,-0.153554,-0.083830,-0.328947,0.012544,0.018974,-0.122389,0.181968
AveRooms,0.325226,-0.153554,1.000000,0.844929,-0.064918,-0.002695,0.098234,-0.006268,0.121449
AveBedrms,-0.049465,-0.083830,0.844929,1.000000,-0.054677,-0.004904,0.065253,0.016482,-0.047031
Population,-0.033198,-0.328947,-0.064918,-0.054677,1.000000,0.072681,-0.080496,0.082442,-0.068364
AveOccup,0.025034,0.012544,-0.002695,-0.004904,0.072681,1.000000,0.009939,-0.002767,-0.024653
Latitude,-0.063649,0.018974,0.098234,0.065253,-0.080496,0.009939,1.000000,-0.932859,-0.108567
Longitude,-0.006102,-0.122389,-0.006268,0.016482,0.082442,-0.002767,-0.932859,1.000000,-0.066755
avg_price,0.641759,0.181968,0.121449,-0.047031,-0.068364,-0.024653,-0.108567,-0.066755,1.000000


In [8]:
observed_data['Population'] = np.log(observed_data['Population'])

x_cols = ['MedInc', 'MedInc', 'AveBedrms', 'Population', 'AveOccup']
X_train = observed_data[x_cols]
y_train = observed_data['avg_price']

X_test = unobserved_data[x_cols]
y_test = unobserved_data['avg_price']

In [9]:
from sklearn.linear_model import LinearRegression

# Create a Linear Regression model
model = LinearRegression()

# Train the model using the training data
model.fit(X_train, y_train)

LinearRegression()

In [10]:
from sklearn.metrics import mean_squared_error, r2_score

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

Mean Squared Error (MSE): 3282.67
R-squared (R²): -6629.65
